In [71]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.stats import entropy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import socket, struct

import imblearn
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import NearMiss
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import class_weight
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from scipy.spatial.distance import jensenshannon
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


In [83]:
class BayesianRandomForest:
    def __init__(self, n_estimators=100, random_state=42):
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.estimators = []
        
    def fit(self, X, y):
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(X.shape[0], X.shape[0], replace=True)
            X_bootstrap = X[bootstrap_indices]
            y_bootstrap = y[bootstrap_indices]
            estimator = RandomForestClassifier(n_estimators=1, random_state=self.random_state)
            estimator.fit(X_bootstrap, y_bootstrap)
            self.estimators.append(estimator)
    
    def predict_proba(self, X):
        predictions = np.array([estimator.predict_proba(X) for estimator in self.estimators])
        return np.mean(predictions, axis=0)

class AINetworkTrafficClassifier:
    def __init__(self, n_estimators=100, random_state=42):
        self.classifier = BayesianRandomForest(n_estimators=n_estimators, random_state=random_state)
        self.known_classes = None
        self.instability_threshold = 0.5
        self.familiarity_threshold = 0.5
        self.label_encoder = LabelEncoder()
        
    def fit(self, X, y):
        self.known_classes = np.unique(y)
        self.label_encoder.fit(self.known_classes)
        y_encoded = self.label_encoder.transform(y)
        self.classifier.fit(X, y_encoded)
    
    def predict(self, X):
        probas = self.classifier.predict_proba(X)
        return self.label_encoder.inverse_transform(np.argmax(probas, axis=1))
    
    def compute_instability(self, X):
        proba1 = self.classifier.predict_proba(X)
        proba2 = self.classifier.predict_proba(X)
        print(proba1)
        print(proba2)
        instability = np.mean([jensenshannon(p1, p2) for p1, p2 in zip(proba1, proba2)])
        return instability
    
    def compute_familiarity(self, X):
        probas = self.classifier.predict_proba(X)
        confidence = np.max(probas, axis=1)
        instability = self.compute_instability(X)
        familiarity = confidence - instability
        return familiarity
    
    def filter_unknown_packets(self, X):
        familiarity = self.compute_familiarity(X)
        return X[familiarity < self.familiarity_threshold]
    
    def update_model(self, X_new, y_new):
        X_combined = np.vstack([self.classifier.estimators_[0].tree_.X, X_new])
        y_combined = np.hstack([self.classifier.classes_, y_new])
        self.fit(X_combined, y_combined)

In [84]:
data=pd.read_csv('1st.csv',header=None)

In [85]:
columns = [
    'duration', 'min_fiat', 'min_biat', 'max_fiat', 'max_biat', 'mean_fiat', 'mean_biat',
    'std_fiat', 'std_biat', 'flowPktsPerSecond', 'flowBytesPerSecond', 'min_flowiat',
    'max_flowiat', 'mean_flowiat', 'std_flowiat', 'min_active', 'mean_active', 'max_active',
    'std_active', 'min_idle', 'mean_idle', 'max_idle', 'std_idle', 'class1'
]
data.columns = columns

# Inspect the data
data.head()

,duration,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,std_fiat,std_biat,flowPktsPerSecond,...,std_flowiat,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,class1
0,76,-1,-1,-1,-1,0.0,0.0,0.0,0.0,26315.789474,...,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,CHAT
1,54,-1,-1,-1,-1,0.0,0.0,0.0,0.0,37037.037037,...,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,CHAT
2,67,-1,-1,-1,-1,0.0,0.0,0.0,0.0,29850.746269,...,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,CHAT
3,75,-1,-1,-1,-1,0.0,0.0,0.0,0.0,26666.666667,...,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,CHAT
4,14512571,14512571,-1,14512571,-1,14500000.0,0.0,0.0,0.0,0.206717,...,10300000.0,14512571,14500000.0,14512571,0.0,14512502,14500000.0,14512502,0.0,CHAT


In [86]:
# for column in data.columns:
#     # Calculate the mean of the numeric values in the column
#     mean_value = pd.to_numeric(data[column], errors='coerce').mean()
    
#     # Replace non-numeric values with the calculated mean
#     data[column] = pd.to_numeric(data[column], errors='coerce').fillna(mean_value)

In [87]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [88]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode the labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3)

In [90]:
classifier = AINetworkTrafficClassifier()
classifier.fit(X_train, y_train)

In [91]:
y_pred = classifier.predict(X_test)
initial_accuracy = accuracy_score(y_test, y_pred)
print(f"Initial accuracy: {initial_accuracy}")

Initial accuracy: 0.8625134264232008


In [94]:
X_new = X_test[:1]  # Assume first 100 samples are new traffic
y_new = y_test[:1]

In [95]:
instability = classifier.compute_instability(X_new)
print(f"Instability: {instability}")

if instability > classifier.instability_threshold:
    print("Model update triggered")
    
    X_unknown = classifier.filter_unknown_packets(X_new)
    
    y_unknown = y_new[classifier.compute_familiarity(X_new) < classifier.familiarity_threshold]
    
    # Update the model
    classifier.update_model(X_unknown, y_unknown)
    
    # Evaluate updated performance
    y_pred_updated = classifier.predict(X_test)
    updated_accuracy = accuracy_score(y_test, y_pred_updated)
    print(f"Updated accuracy: {updated_accuracy}")
    
    # Print confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_updated)
    print("Confusion Matrix:")
    print(conf_matrix)
    
    # Save results to CSV
    results = pd.DataFrame({
        'True Label': y_test,
        'Predicted Label': y_pred_updated,
        'Familiarity Score': classifier.compute_familiarity(X_test)
    })
    results.to_csv('classification_results.csv', index=False)
else:
    print("No update needed")

[[0.03 0.   0.81 0.   0.15 0.01 0.  ]]
[[0.03 0.   0.81 0.   0.15 0.01 0.  ]]
Instability: 0.0
No update needed
